<a id='sec0'></a>
# Predictive annotation of quantification methods with Noun Chunks 3
It appears there are some weird noun chunks and here I'm re-acessing the vocabulary, so that improved text-processing scheme can be established.<br>
- <a href='#sec1'><b></b></a>
- <a href='#sec2'><b></b></a>
- <a href='#sec3'><b></b></a>
- <a href='#sec4'><b></b></a>

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import spacy
import re

from nltk.corpus import stopwords
from gensim import corpora, models
from gensim import matutils
import matplotlib.pyplot as plt
%matplotlib inline

<a id='sec1-1'></a>
#### 1. Read in data and process df

In [2]:
with open('nlp15_data/dfs/all_fields_nchunks_df_quant_dummies.pickle', 'rb') as infile_df:
    df = pickle.load(infile_df)
    
my_dictionary = corpora.Dictionary.load('nlp15_data/whole_dictionary_nchunks.dict')
print(my_dictionary)

Dictionary(125116 unique tokens: [') luminal a tumors', '1 tumors', '24 lymph node', '24 lymph node negative grade 1 luminal a primary breast tumors', '4405 proteins']...)


In [3]:
df.head(2)

,dataset_id,sample_protocol,data_protocol,description,instruments,exp_types,quant_methods,labhead_fullname,silac,ms1_label_free,spectrum_counting,tmt,itraq,label_free
0,PXD000029,"[breast cancer tissue lysates, reduction, alky...","[proteomics data, proteome discoverer, fdr<0.0...","[current prognostic factors, precise risk-disc...",LTQ Orbitrap Velos,Shotgun proteomics,itraq,Pavel Bouchal,0,0,0,0,1,0
1,PXD000164,"[protein extraction, catheter biofilm small pi...","[tryptic digest, reversed phase, rp) chromatog...","[term-catheterization, catheter-associated bac...",LTQ Orbitrap Velos,Shotgun proteomics,label free,Katharina Riedel,0,0,0,0,0,1


In [4]:
tokens = sorted(list(my_dictionary.token2id.keys()))
len(tokens)

125116

In [5]:
for token in tokens[:200]:
    print(token)

0.1
1.9
2.2
3.5
cn
" molecular & cellular proteomics
""cessation
""heavy"" media
""light
""lock mass"" function
""mouse complete proteome
""switch
"071812_glit3t
"ab initio
"annotation matrix
"b
"background
"bait
"complete" t. cruzi proteome
"database
"deep" proteome
"experimental design proteomexchange.txt
"experiments
"filter assisted sample preparation
"gene name & annotations" section
"global normalization
"good outcome
"heavy" amino-acids
"heavy" capture assay samples
"heavy" deuterated versions
"heavy" isotope-labeled amino acids
"heavy" media
"input" material
"large-scale targeted proteomics
"light
"light" l-arginine and l-lysine
"light" media
"light" silac media
"lock mass" function
"match
"medium-heavy" and "heavy" cells
"mixo
"mouse complete proteome
"multicomp" package
"neg
"negative" weight acetylation
"poor outcome
"precursor ion quantifier" node
"protein identification and network analysis" methods section
"proteinpilot mgf" parameters
"proteomic ruler" method
"pseud

In [6]:
regex1 = r'^[^A-Za-z]+'
sorted([re.sub(regex1, '', s).strip() for s in tokens[:200]])

['',
 '',
 '',
 '',
 '',
 '',
 'a/%b',
 'ab initio',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acn',
 'acn',
 'and (-) manassantin a samples',
 'annotation matrix',
 'aqueous 2% acetonitrile',
 'b',
 'b',
 'b',
 'background',
 'bait',
 "bimodalitytest' package",
 'bpa) exposure',
 'c10416',
 'c2h6sio)6',
 'cessation',
 'cn',
 'complete" t. cruzi proteome',
 'confluence',
 'confluency',
 'contaminants',
 'control',
 'control sample',
 "control' ratios",
 'cor',
 "core' human pr",
 'database',
 'deep" proteome',
 'deuteration',
 'di)methylated hla class',
 'dmso',
 'dmso',
 "earliest' fraction",
 'early risers',
 'es803',
 'experimental design proteomexchange.txt',
 'experiments',
 'f4799',
 'filter assisted sample preparation',
 'formic acid',
 'formic acid) b (80% acetonitrile',
 'forward and reverse) experiment',
 'fractions',
 "gel-top' approach",
 'gene name & annotations" section',
 'glit3t',
 'global normalization',
 'g

In [7]:
regex2 = r'\'|\"|\(.+\)'
tokens2 = sorted([re.sub(regex1, '', s).strip() for s in tokens[:200]])
[re.sub(regex2, '', s).strip() for s in tokens2]

['',
 '',
 '',
 '',
 '',
 '',
 'a/%b',
 'ab initio',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acn',
 'acn',
 'and  manassantin a samples',
 'annotation matrix',
 'aqueous 2% acetonitrile',
 'b',
 'b',
 'b',
 'background',
 'bait',
 'bimodalitytest package',
 'bpa) exposure',
 'c10416',
 'c2h6sio)6',
 'cessation',
 'cn',
 'complete t. cruzi proteome',
 'confluence',
 'confluency',
 'contaminants',
 'control',
 'control sample',
 'control ratios',
 'cor',
 'core human pr',
 'database',
 'deep proteome',
 'deuteration',
 'di)methylated hla class',
 'dmso',
 'dmso',
 'earliest fraction',
 'early risers',
 'es803',
 'experimental design proteomexchange.txt',
 'experiments',
 'f4799',
 'filter assisted sample preparation',
 'formic acid',
 'formic acid) b (80% acetonitrile',
 'forward and reverse) experiment',
 'fractions',
 'gel-top approach',
 'gene name & annotations section',
 'glit3t',
 'global normalization',
 'globin',
 'g

In [8]:
regex3 = r'^[^A-Za-z]+|\'|\"|\(.+\)'
tokens3 = sorted([re.sub(regex3, '', s).strip() for s in tokens[:200]])

In [9]:
tokens3

['',
 '',
 '',
 '',
 '',
 '',
 'a/%b',
 'ab initio',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acn',
 'acn',
 'and  manassantin a samples',
 'annotation matrix',
 'aqueous 2% acetonitrile',
 'b',
 'b',
 'b',
 'background',
 'bait',
 'bimodalitytest package',
 'bpa) exposure',
 'c10416',
 'c2h6sio)6',
 'cessation',
 'cn',
 'complete t. cruzi proteome',
 'confluence',
 'confluency',
 'contaminants',
 'control',
 'control ratios',
 'control sample',
 'cor',
 'core human pr',
 'database',
 'deep proteome',
 'deuteration',
 'di)methylated hla class',
 'dmso',
 'dmso',
 'earliest fraction',
 'early risers',
 'es803',
 'experimental design proteomexchange.txt',
 'experiments',
 'f4799',
 'filter assisted sample preparation',
 'formic acid',
 'formic acid) b (80% acetonitrile',
 'forward and reverse) experiment',
 'fractions',
 'gel-top approach',
 'gene name & annotations section',
 'glit3t',
 'global normalization',
 'globin',
 'g

In [10]:
regex4 = r'^.*\)\W*|\(.*$'
tokens4 = sorted([re.sub(regex4, '', s).strip() for s in tokens3])

In [11]:
tokens4

['',
 '',
 '',
 '',
 '',
 '',
 '6',
 'a/%b',
 'ab initio',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acn',
 'acn',
 'and  manassantin a samples',
 'annotation matrix',
 'aqueous 2% acetonitrile',
 'b',
 'b',
 'b',
 'b',
 'background',
 'bait',
 'bimodalitytest package',
 'c10416',
 'cessation',
 'cn',
 'complete t. cruzi proteome',
 'confluence',
 'confluency',
 'contaminants',
 'control',
 'control ratios',
 'control sample',
 'conversion',
 'cor',
 'core human pr',
 'database',
 'deep proteome',
 'deuteration',
 'dmso',
 'dmso',
 'earliest fraction',
 'early risers',
 'endoplasmic reticulum',
 'es803',
 'experiment',
 'experimental design proteomexchange.txt',
 'experiments',
 'exposure',
 'exposure',
 'f4799',
 'filter assisted sample preparation',
 'former smoker group',
 'formic acid',
 'fractions',
 'gel-top approach',
 'gene name & annotations section',
 'glit3t',
 'global normalization',
 'globin',
 'good outcome',
 

In [12]:
regex5 = r'^.$|.*=.*|^\w |^mm '
tokens5 = sorted([re.sub(regex5, '', s) for s in tokens4])

In [13]:
[s for s in tokens5 if s != '']

['a/%b',
 'ab initio',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acetonitrile',
 'acn',
 'acn',
 'ammonium formate',
 'ammonium formate',
 'ammonium hydroxide',
 'and  manassantin a samples',
 'annotation matrix',
 'aqueous 2% acetonitrile',
 'background',
 'bait',
 'bimodalitytest package',
 'c10416',
 'cessation',
 'cn',
 'complete t. cruzi proteome',
 'confluence',
 'confluency',
 'contaminants',
 'control',
 'control ratios',
 'control sample',
 'conversion',
 'cor',
 'core human pr',
 'database',
 'deep proteome',
 'deuteration',
 'dmso',
 'dmso',
 'earliest fraction',
 'early risers',
 'endoplasmic reticulum',
 'es803',
 'experiment',
 'experimental design proteomexchange.txt',
 'experiments',
 'exposure',
 'exposure',
 'f4799',
 'filter assisted sample preparation',
 'former smoker group',
 'formic acid',
 'fractions',
 'gel-top approach',
 'gene name & annotations section',
 'glit3t',
 'global normalization',
 'globin',
 'good outcom

In [14]:
def extract_noun_chuncks2(tokens):
    # Remove quote marks and closed brackets and clauses in half brackets
    regex1 = r'^[^A-Za-z]+|\'|\"|\(.+\)'
    tokens = [re.sub(regex1, '', s).strip() for s in tokens]
    
    # Remove half open brackets
    regex2 = r'^.*\)\W*|\(.*$'
    tokens = [re.sub(regex2, '', s).strip() for s in tokens]
    
    # Remove one letter words, ones that start with one latter and a space,
    # and reminiscence of mM
    regex3 = r'^.$|.*=.*|^\w |^mm '
    tokens = [re.sub(regex3, '', s).strip() for s in tokens]
    
    tokens = [s for s in tokens if s != '']
    
    return tokens

In [15]:
extract_noun_chuncks2(tokens[:200])

['cn',
 'molecular & cellular proteomics',
 'cessation',
 'heavy media',
 'light',
 'lock mass function',
 'mouse complete proteome',
 'switch',
 'glit3t',
 'ab initio',
 'annotation matrix',
 'background',
 'bait',
 'complete t. cruzi proteome',
 'database',
 'deep proteome',
 'experimental design proteomexchange.txt',
 'experiments',
 'filter assisted sample preparation',
 'gene name & annotations section',
 'global normalization',
 'good outcome',
 'heavy amino-acids',
 'heavy capture assay samples',
 'heavy deuterated versions',
 'heavy isotope-labeled amino acids',
 'heavy media',
 'input material',
 'large-scale targeted proteomics',
 'light',
 'light l-arginine and l-lysine',
 'light media',
 'light silac media',
 'lock mass function',
 'match',
 'medium-heavy and heavy cells',
 'mixo',
 'mouse complete proteome',
 'multicomp package',
 'neg',
 'negative weight acetylation',
 'poor outcome',
 'precursor ion quantifier node',
 'protein identification and network analysis methods 

In [16]:
%%time
with open('nlp15_data/dfs/all_fields_nchunks_df_quant_dummies.pickle', 'rb') as infile_df:
    df = pickle.load(infile_df)
    
# extrac noun_chunks from the text
if True:
    print('Processing sample_protocol...')
    df.loc[:, 'sample_protocol'] = df.loc[:, 'sample_protocol'].apply(lambda x: extract_noun_chuncks2(x))
    
    print('Processing data_protocol...')
    df.loc[:, 'data_protocol'] = df.loc[:, 'data_protocol'].apply(lambda x: extract_noun_chuncks2(x))
    
    print('Processing description...')
    df.loc[:, 'description'] = df.loc[:, 'description'].apply(lambda x: extract_noun_chuncks2(x))

Processing sample_protocol...
Processing data_protocol...
Processing description...
CPU times: user 1.61 s, sys: 11.5 ms, total: 1.62 s
Wall time: 1.62 s


In [17]:
df.head()

,dataset_id,sample_protocol,data_protocol,description,instruments,exp_types,quant_methods,labhead_fullname,silac,ms1_label_free,spectrum_counting,tmt,itraq,label_free
0,PXD000029,"[breast cancer tissue lysates, reduction, alky...","[proteomics data, proteome discoverer, fdr<0.0...","[current prognostic factors, precise risk-disc...",LTQ Orbitrap Velos,Shotgun proteomics,itraq,Pavel Bouchal,0,0,0,0,1,0
1,PXD000164,"[protein extraction, catheter biofilm small pi...","[tryptic digest, reversed phase, chromatograph...","[term-catheterization, catheter-associated bac...",LTQ Orbitrap Velos,Shotgun proteomics,label free,Katharina Riedel,0,0,0,0,0,1
2,PXD000175,"[generation, mdia2-based immunocomplexes and m...","[ms data, peak lists, raw data files, proteome...","[mdia2, auto-inhibited formin, actin dynamics,...",LTQ Orbitrap,Affinity purification coupled with mass spectr...,spectrum counting,Maarten Altelaar,0,0,1,0,0,0
3,PXD000376,"[naïve cd4+ t cells, magnetic beads, manufactu...","[ms data, proteome discoverer, thermo fisher s...","[technological advances, genomics, epigenomics...","LTQ Orbitrap Velos, LTQ Orbitrap Elite",Shotgun proteomics,itraq,Akhilesh Pandey,0,0,0,0,1,0
4,PXD000397,"[cells, hours, serum, free dmem, phenol red, e...","[raw data, maxquant, results, scripts, house, ...","[project, expression patterns, hypoxia, secret...",Q Exactive,Shotgun proteomics,silac,Janine T Erler,1,0,0,0,0,0


In [18]:
df.sample_protocol.iloc[0]

['breast cancer tissue lysates',
 'reduction',
 'alkylation',
 'methyl-methanethiosulfonate trypsin digestion',
 'plex labeling',
 'hilic fractionation',
 'lc-ms/ms',
 'orbitrap velos',
 'transcriptomics',
 'immunohistochemistry',
 'detailed description',
 'manuscript']

In [19]:
# Serialize and save the processed DF
if True:
    with open('nlp15_data/dfs/all_fields_nchunks_df_quant_dummies2.pickle', 'wb') as out_df:
        pickle.dump(df, out_df)

In [20]:
# Save the corpora
# This time I'll only use combined protocols and that combined with description field (i.e. two corpora) as in nlp13
if True:
    # Protocols combined
    protocols_corpus = list(df.sample_protocol + df.data_protocol)
    with open('nlp15_data/corpora/protocols_corpus_nchunks2.pickle', 'wb') as outfile:
        pickle.dump(protocols_corpus, outfile)

    # All combined
    whole_corpus = list(df.sample_protocol + df.data_protocol + df.description)
    with open('nlp15_data/corpora/whole_corpus_nchunks2.pickle', 'wb') as outfile:
        pickle.dump(whole_corpus, outfile)

In [21]:
# Create and save dictionary using whole_corpus
if True:
    my_dictionary = corpora.Dictionary(whole_corpus)
    my_dictionary.save('nlp15_data/whole_dictionary_nchunks2.dict')

In [22]:
# Save BoW and Tf-Idf
if True:
    # BoW transformations and save
    protocols_bow = [my_dictionary.doc2bow(text) for text in protocols_corpus]
    whole_bow = [my_dictionary.doc2bow(text) for text in whole_corpus]
    corpora.MmCorpus.serialize('nlp15_data/bow_and_tfidf/protocols_bow_nchunks2.mm', protocols_bow)
    corpora.MmCorpus.serialize('nlp15_data/bow_and_tfidf/whole_bow_nchunks2.mm', whole_bow)
    
    # Tf-Idf transformations  and save
    protocols_tfidf_model = models.TfidfModel(protocols_bow)
    protocols_tfidf = protocols_tfidf_model[protocols_bow]
    corpora.MmCorpus.serialize('nlp15_data/bow_and_tfidf/protocols_tfidf_nchunks2.mm', protocols_tfidf)
    
    whole_tfidf_model = models.TfidfModel(whole_bow)
    whole_tfidf = whole_tfidf_model[whole_bow]
    corpora.MmCorpus.serialize('nlp15_data/bow_and_tfidf/whole_tfidf_nchunks2.mm', whole_tfidf)